# Postprocessing for 2023 Internal Lysis paper

In [ ]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=110,
    target_version=black.TargetVersion.PY310,
)

In [ ]:
import datetime
import math
import os
import pickle
import re

from enum import IntEnum, unique

import scipy
from tqdm.notebook import trange, tqdm, tnrange

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

from matplotlib.animation import FFMpegWriter, FuncAnimation
from matplotlib.colors import BoundaryNorm, ListedColormap

import lysis

pd.reset_option("display.precision")
pd.set_option("display.float_format", lambda x: f"{x:,.3f}")

In [ ]:
@unique
class MolStatus(IntEnum):
    UNBOUND = 0
    BOUND = 1
    MACRO_UNBOUND = 2
    MICRO_UNBOUND = 3

In [ ]:
scenario_type = np.dtype(
    [
        ("descriptor", np.str_, 40),
        ("total_molecules", int),
        ("pore_size", float),  # in microns. Code requires cm
        ("fiber_diameter", float),  # in nanometers. Needs to be added to pore_size to get grid_node_distance
        ("cols", int),
        ("rows", int),
        ("empty_rows", int),
        ("forced_unbind", float),
    ]
)
mechanism_type = np.dtype([("descriptor", np.str_, 40), ("executable", np.str_, 40)])
run_type = np.dtype(
    [
        ("exp_code", np.str_, 15),
        ("scenario", np.str_, 40),
        ("mechanism", np.str_, 40),
        ("seed", int),
        ("running_time", int),
        ("save_interval", int),
    ]
)
diameter_code = {
    72.7: "Q2",
    145.4: "Q4",
}

In [ ]:
# seed sequence entropy: 3881821051554698152964433817123076384
scenarios = np.array(
    [
        ("TN-L_9350", 9_350, 1.0135, 72.7, 93, 93, 0, 0.0852),
        ("TN-L_307", 307, 1.0135, 72.7, 93, 93, 0, 0.0852),
        ("TN-D_684", 684, 0.22, 72.7, 342, 116, 0, 0.0852),
        ("TN-D_9350", 9_350, 0.22, 72.7, 342, 116, 0, 0.0852),
        ("TN-D_307", 307, 0.22, 72.7, 342, 116, 0, 0.0852),
        ("TK-L_3042", 3_042, 1.0135, 145.4, 87, 29, 0, 0.0729129),
        ("TK-L_9350", 9_350, 1.0135, 145.4, 87, 29, 0, 0.0729129),
        ("TK-L_307", 307, 1.0135, 145.4, 87, 29, 0, 0.0729129),
        ("TK-D_307", 307, 0.22, 145.4, 274, 42, 0, 0.0729129),
        ("TK-D_9350", 9350, 0.22, 145.4, 274, 42, 0, 0.0729129),
    ],
    dtype=scenario_type,
)
mechanisms = np.array(
    [
        ("Into and along - Internal", "macro_diffuse_into_and_along__internal"),
    ],
    dtype=mechanism_type,
)

In [ ]:
in_file_code = "_PLG2_tPA01_{input_code}.dat"
out_file_code = "_{scenario_code}"

# slope_tolerance = 1e-3
rng = np.random.default_rng(65463453)

In [ ]:
group_code = "2023-06-09-10"
runs = np.array(
    [
        (group_code + "00", "TN-L_9350", "Into and along - Internal", 1_034_836_197, 0, 10),
        (group_code + "01", "TN-L_307", "Into and along - Internal", 2_464_079_704, 0, 100),
        (group_code + "02", "TN-D_684", "Into and along - Internal", 982_919_027, 0, 100),
        (group_code + "03", "TN-D_9350", "Into and along - Internal", 129_314_213, 0, 10),
        (group_code + "04", "TN-D_307", "Into and along - Internal", 2_705_873_059, 0, 100),
        (group_code + "05", "TK-L_3042", "Into and along - Internal", 2_104_158_012, 0, 10),
        (group_code + "06", "TK-L_9350", "Into and along - Internal", 2_608_028_918, 0, 10),
        (group_code + "07", "TK-L_307", "Into and along - Internal", 1_819_293_232, 0, 100),
        (group_code + "08", "TK-D_307", "Into and along - Internal", 3_128_495_922, 0, 100),
        (group_code + "09", "TK-D_9350", "Into and along - Internal", 106_672_317, 0, 10),
    ],
    dtype=run_type,
)

In [ ]:
# Get default parameters
e = lysis.Experiment(os.path.join("..", "..", "data"), experiment_code="0000-00-00-0000")
e.initialize_macro_param()
df = pd.DataFrame([e.macro_params])

# Create empty DataFrame using the scenario names as rows and parameter names as columns
index = pd.Index(runs["scenario"], name="Scenario")
parameters_df = pd.DataFrame(index=index, columns=df.columns)

In [ ]:
formats = {
    "binding_rate": "{:.2f}",
    "pore_size": "{:1.2e}",
    "diffusion_coeff": "{:1.2e}",
    # 'binding_sites': "",
    "forced_unbind": "{:.3f}",
    "average_bind_time": "{:.2f}",
    "grid_node_distance": "{:.3f}",
    # 'cols', 'rows', 'fiber_rows', 'empty_rows', 'last_empty_edge',
    "full_row": "{:,}",
    "xz_row": "{:,}",
    "total_edges": "{:,}",
    "total_fibers": "{:,}",
    "total_molecules": "{:,}",
    "moving_probability": "{:.2f}",
    "microscale_runs": "{:,}",
    # 'total_trials',
    "total_time": "{:,.0f}",
    "time_step": "{:1.3e}",
    "total_time_steps": "{:,}",
    "seed": "{:,}",
    # 'save_interval',
    # 'number_of_saves',
}
cols_to_drop = [
    "state",
    "input_data",
    "output_data",
    "log_lvl",
    "duplicate_fortran",
    "processing_library",
    "macro_version",
]
# ,

In [ ]:
index = pd.Index(runs["scenario"], name="Scenario")
degrade_percent_markers = [0.05, 0.2, 0.5, 0.8]
columns = pd.MultiIndex.from_product(
    [[f"{int(i*100)}%" for i in degrade_percent_markers], range(10)], names=["Percent Degraded", "Simulation"]
)
degradation_time_df = pd.DataFrame(index=index, columns=columns)

degrade_slope_between_percent = [[0.2, 0.8], [0.2, 0.5], [0.5, 0.8]]
columns = pd.MultiIndex.from_product(
    [[f"{int(i[0]*100)}% to {int(i[1]*100)}%" for i in degrade_slope_between_percent], range(10)],
    names=["Degradation Interval", "Simulation"],
)
slopes_df = pd.DataFrame(index=index, columns=columns)
binds_df = pd.DataFrame(index=index, columns=pd.Index(range(10), name="Simulation"))

mol_time_statistics = [
    "Total Binds",
]
mol_time_df = pd.DataFrame(index=index, columns=mol_time_statistics)

pore_stat_df = pd.DataFrame(index=index, columns=[f"{int(i*100)}%" for i in degrade_percent_markers])

In [ ]:
# For a light colormap of degraded fibers
colormap_f = plt.cm.cividis(np.arange(plt.cm.cividis.N))
colormap_f[:, 3] = 0.3
# For very light grey degraded fibers
colormap_f = ["xkcd:light grey"]

colormap_f = ListedColormap(colormap_f)
colormap_f.set_extremes(bad="xkcd:blue", under="white", over="xkcd:blue")
colormap_m = ListedColormap(["xkcd:green", "xkcd:cyan"])
bounds = [0, 0.5, 1]
norm_m = BoundaryNorm(bounds, colormap_m.N)

angle = (1 / 2 - 1 / 8) * math.pi
radius = 0.5

In [ ]:
def load_fortran_files(exp, file_code):
    n_save = np.asarray(
        [
            np.fromfile(
                os.path.join(e.os_path, f"{sim:02}", f"Nsave{file_code[:-4]}_{sim:02}{file_code[-4:]}"),
                dtype=np.int32,
            )[0]
            for sim in range(e.macro_params.total_trials)
        ]
    )
    n_save += 1
    n_save

    tsave = [
        np.fromfile(os.path.join(e.os_path, f"{sim:02}", f"tsave{file_code[:-4]}_{sim:02}{file_code[-4:]}"))
        for sim in range(e.macro_params.total_trials)
    ]
    tmax = max([np.max(arr) for arr in tsave])

    mfpt = np.asarray(
        [
            np.fromfile(
                os.path.join(e.os_path, f"{sim:02}", f"mfpt{file_code[:-4]}_{sim:02}{file_code[-4:]}")
            )
            for sim in range(e.macro_params.total_trials)
        ]
    )

    mol_location = []
    mol_status = []
    mapped_deg = []
    for sim in range(e.macro_params.total_trials):
        raw_mol_location = np.fromfile(
            os.path.join(e.os_path, f"{sim:02}", f"m_loc{file_code[:-4]}_{sim:02}{file_code[-4:]}"),
            dtype=np.int32,
        )
        mol_location.append(raw_mol_location.reshape(n_save[sim], e.macro_params.total_molecules) - 1)

        raw_mol_status = np.fromfile(
            os.path.join(e.os_path, f"{sim:02}", f"m_bound{file_code[:-4]}_{sim:02}{file_code[-4:]}"),
            dtype=np.int32,
        )
        raw_mol_status = raw_mol_status.astype(np.bool_)
        mol_status.append(raw_mol_status.reshape(n_save[sim], e.macro_params.total_molecules))

        raw_deg = np.fromfile(
            os.path.join(e.os_path, f"{sim:02}", f"f_deg_time{file_code[:-4]}_{sim:02}{file_code[-4:]}")
        )
        # raw_mapped_deg[raw_deg == 0] = tmax + e.macro_params.save_interval  # float('inf') #
        mapped_deg.append(raw_deg.reshape(n_save[sim], e.macro_params.total_edges))

    return n_save, mapped_deg, tsave, mfpt, mol_location, mol_status


def find_degraded_fraction(exp, deg, tsave):
    degraded_fraction = []
    for r in range(exp.macro_params.total_trials):
        run_degraded_fraction = np.empty(deg[r].shape[0], dtype=np.float_)
        for t in range(deg[r].shape[0]):
            run_degraded_fraction[t] = np.count_nonzero(deg[r][t] <= tsave[r][t])
        run_degraded_fraction -= exp.macro_params.empty_rows * exp.macro_params.full_row
        degraded_fraction.append(run_degraded_fraction / exp.macro_params.total_fibers)
    return degraded_fraction


def find_degradation_marker_frames(exp, degraded_fraction):
    degrade_marker_frames = np.empty(
        (exp.macro_params.total_trials, len(degrade_percent_markers)), dtype=np.int_
    )
    for sim in range(exp.macro_params.total_trials):
        for marker in range(len(degrade_percent_markers)):
            degrade_marker_frames[sim, marker] = np.argmax(
                degraded_fraction[sim] >= degrade_percent_markers[marker]
            )
    return degrade_marker_frames


# Changed back to min
def find_degradation_marker_times(exp, degrade_marker_frames, tsave):
    degradation_marker_times = np.empty(
        (exp.macro_params.total_trials, len(degrade_percent_markers)), dtype=np.float_
    )
    for sim in range(exp.macro_params.total_trials):
        degradation_marker_times[sim] = tsave[sim][degrade_marker_frames[sim]]
    return degradation_marker_times / 60


def degradation_rates(exp, degrade_marker_frames, degraded_fraction, tsave):
    degradation_rates = np.empty(
        (exp.macro_params.total_trials, len(degrade_slope_between_percent)), dtype=np.float_
    )
    for sim in range(exp.macro_params.total_trials):
        for slope in range(len(degrade_slope_between_percent)):
            start_frame = degrade_marker_frames[
                sim, degrade_percent_markers.index(degrade_slope_between_percent[slope][0])
            ]
            end_frame = degrade_marker_frames[
                sim, degrade_percent_markers.index(degrade_slope_between_percent[slope][1])
            ]
            delta_y = degraded_fraction[sim][end_frame] - degraded_fraction[sim][start_frame]
            delta_x = tsave[sim][end_frame] - tsave[sim][start_frame]
            degradation_rates[sim, slope] = delta_y / delta_x * 60  # * 100
    return degradation_rates


# Must match units
# Is expecting degradation rates in frac/min
def plot_degradation_percent(exp, degraded_fraction, tsave, degrade_marker_frames, degradation_rates):
    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_xlim(0, (max([np.max(deg_run[-1, :]) for deg_run in deg]) // 60) + 1)
    ax.set_ylim(-0.1, 1.1)
    for sim in range(exp.macro_params.total_trials):
        plt.plot(tsave[sim] / 60, degraded_fraction[sim])
        for slope in range(len(degrade_slope_between_percent)):
            start_frame = degrade_marker_frames[
                sim, degrade_percent_markers.index(degrade_slope_between_percent[slope][0])
            ]
            end_frame = degrade_marker_frames[
                sim, degrade_percent_markers.index(degrade_slope_between_percent[slope][1])
            ]
            lin_func = (
                lambda x: degradation_rates[sim, slope] * (x - tsave[sim][start_frame] / 60)
                + degraded_fraction[sim][start_frame]
            )
            plt.plot(
                tsave[sim][start_frame:end_frame] / 60,
                lin_func(tsave[sim][start_frame:end_frame] / 60),
                color="b",
                alpha=0.5,
                zorder=0.1,
            )
    fig.savefig(os.path.join(exp.os_path, "deg_rate" + file_code[:-4] + ".png"), bbox_inches="tight")
    plt.close()


def plot_coords(i, j):
    x = j
    y = -i
    if j % 3 == 0:
        return x / 3.0, y - 0.5
    if j % 3 == 1:
        return (x - 1) / 3.0, y
    if j % 3 == 2:
        return (x - 2) / 3.0 + 0.5, y


def get_edge_index(exp):
    edge_index = np.empty(exp.macro_params.total_edges, dtype=tuple)
    for k in range(exp.macro_params.total_edges):
        edge_index[k] = lysis.from_fortran_edge_index(k, exp.macro_params.rows, exp.macro_params.cols)
    return edge_index


def get_rev_edge_index(exp):
    rev_edge_index = np.full((exp.macro_params.rows, exp.macro_params.full_row), -1, dtype=int)
    for k in range(exp.macro_params.total_edges):
        rev_edge_index[lysis.from_fortran_edge_index(k, exp.macro_params.rows, exp.macro_params.cols)] = k
    return rev_edge_index


def animation_data(exp, edge_index, mol_location):
    x_f = np.empty(exp.macro_params.total_edges, dtype=float)
    y_f = np.empty(exp.macro_params.total_edges, dtype=float)
    for k in range(exp.macro_params.total_edges):
        i, j = edge_index[k]
        x_f[k], y_f[k] = plot_coords(i, j)
    d_x = (rng.random(size=exp.macro_params.total_molecules) - 0.5) / 2.5
    d_y = (rng.random(size=exp.macro_params.total_molecules) - 0.5) / 2.5
    x_m = []
    y_m = []
    for r in range(exp.macro_params.total_trials):
        x = np.empty((n_save[r], exp.macro_params.total_molecules), dtype=np.float_)
        y = np.empty((n_save[r], exp.macro_params.total_molecules), dtype=np.float_)
        for t in range(n_save[r]):
            for k in range(exp.macro_params.total_molecules):
                i, j = edge_index[mol_location[r][t, k]]
                x[t, k], y[t, k] = plot_coords(i, j)
            x[t] += d_x
            y[t] += d_y
        x_m.append(x)
        y_m.append(y)
    return x_f, y_f, x_m, y_m


def create_animation(exp, file_code, edge_index, deg, mol_status, tsave, x_f, y_f, x_m, y_m):
    for sim in range(1):
        fig = plt.figure(figsize=(exp.macro_params.cols / 10, exp.macro_params.rows / 10), dpi=100)
        ax = fig.add_axes([0, 0, 1, 1])
        ax.set_axis_off()
        vmin = 0
        vmax = exp.macro_params.cols - 1

        ydiff = exp.macro_params.rows - exp.macro_params.cols

        ax.set_xlim(vmin - 0.25, vmax + 0.25)
        ax.set_ylim(-vmax - ydiff - 0.25, -vmin + 0.25)
        ax.set_aspect("equal")
        fig.canvas.draw()
        title = ax.annotate(
            f"Time elapsed: {str(datetime.timedelta(seconds=tsave[sim][0])).split('.')[0]}",
            (0, 0),
            zorder=100,
            va="top",
        )
        s = (ax.get_window_extent().width / (vmax - vmin + 0.5) * 72 / (2 * fig.dpi)) ** 2
        scatt_f = ax.scatter(
            x_f,
            y_f,
            s=s,
            marker="s",
            linewidths=0,
            c=deg[sim][0],
            cmap=colormap_f,
            vmin=exp.macro_params.time_step,
            vmax=exp.macro_params.time_step,
        )
        scatt_m = ax.scatter(
            x_m[sim][0],
            y_m[sim][0],
            s=s / 2.5,
            marker="o",
            linewidths=0,
            c=mol_status[sim][0],
            cmap=colormap_m,
            norm=norm_m,
        )

        def update(frame_number):
            scatt_f.set_array(deg[sim][frame_number])
            if tsave[sim][frame_number] == 0:
                scatt_f.set_clim(vmin=exp.macro_params.time_step, vmax=exp.macro_params.time_step)
            else:
                scatt_f.set_clim(vmin=exp.macro_params.time_step, vmax=tsave[sim][frame_number])
            scatt_m.set_array(mol_status[sim][frame_number])
            scatt_m.set_offsets(np.append((x_m[sim][frame_number],), (y_m[sim][frame_number],), axis=0).T)
            title.set_text(
                f"Time elapsed: {str(datetime.timedelta(seconds=tsave[sim][frame_number])).split('.')[0]}"
            )

        animation = FuncAnimation(fig, update, frames=np.arange(n_save[sim]), interval=200)

        FFwriter = FFMpegWriter(fps=10)
        animation.save(
            os.path.join(e.os_path, f"{sim:02}", f"combined_animation_{sim:02}" + file_code[:-4] + ".mp4"),
            writer=FFwriter,
        )
        plt.close()


def create_animation_stills(
    exp, file_code, degrade_marker_frames, edge_index, deg, mol_status, tsave, x_f, y_f, x_m, y_m
):
    for sim in range(1):
        for i, frame in enumerate(degrade_marker_frames[sim]):
            fig = plt.figure(figsize=(exp.macro_params.cols / 10, exp.macro_params.rows / 10), dpi=100)
            ax = fig.add_axes([0, 0, 1, 1])
            ax.set_axis_off()
            vmin = 0
            vmax = exp.macro_params.cols - 1

            ydiff = exp.macro_params.rows - exp.macro_params.cols

            ax.set_xlim(vmin - 0.25, vmax + 0.25)
            ax.set_ylim(-vmax - ydiff - 0.25, -vmin + 0.25)
            ax.set_aspect("equal")
            fig.canvas.draw()
            title = ax.annotate(
                f"Simulation time elapsed: {str(datetime.timedelta(seconds=tsave[sim][frame])).split('.')[0]}",
                (0, 0),
                zorder=100,
                va="top",
            )
            s = (ax.get_window_extent().width / (vmax - vmin + 0.5) * 72 / (2 * fig.dpi)) ** 2
            scatt_f = ax.scatter(
                x_f,
                y_f,
                s=s,
                marker="s",
                linewidths=0,
                c=deg[sim][frame],
                cmap=colormap_f,
                vmin=exp.macro_params.time_step,
                vmax=tsave[sim][frame],
            )
            scatt_m = ax.scatter(
                x_m[sim][frame],
                y_m[sim][frame],
                s=s / 2.5,
                marker="o",
                linewidths=0,
                c=mol_status[sim][frame],
                cmap=colormap_m,
                norm=norm_m,
            )

            fig.savefig(
                os.path.join(
                    e.os_path,
                    f"{sim:02}",
                    f"experiment_state_plot_r{sim:02}_{degrade_percent_markers[i]:.2f}"
                    + file_code[:-4]
                    + ".png",
                ),
                bbox_inches="tight",
            )
            plt.close()


def save_degrade_fraction_table(exp, file_code, degraded_fraction, tsave):
    max_time = 0
    max_time_idx = None
    for i in range(len(tsave)):
        if len(tsave[i]) > max_time:
            max_time = len(tsave[i])
            max_time_idx = i
    index = pd.Index(range(10), name="Simulation")
    columns = pd.Index((tsave[max_time_idx] / 60), name="Simulation Time Elapsed (min)")
    pd.DataFrame(degraded_fraction, index=index, columns=columns).fillna(1).to_csv(
        os.path.join(exp.os_path, "deg_fraction_at_time_in_min" + file_code[:-4] + ".csv")
    )


def fiber_end_coords(i, j):
    x1 = j // 3
    y1 = i
    if j % 3 == 0:
        x2 = x1
        y2 = y1 + 1
    elif j % 3 == 1:
        x2 = x1 + radius * math.cos(angle)
        y2 = y1 - radius * math.sin(angle)
        x1 = x1 - radius * math.cos(angle)
        y1 = y1 + radius * math.sin(angle)
    elif j % 3 == 2:
        x2 = x1 + 1
        y2 = y1
    return x1, y1, x2, y2


def grid_data(exp, edge_index):
    x1_f = np.empty(exp.macro_params.total_edges, dtype=float)
    y1_f = np.empty(exp.macro_params.total_edges, dtype=float)
    x2_f = np.empty(exp.macro_params.total_edges, dtype=float)
    y2_f = np.empty(exp.macro_params.total_edges, dtype=float)
    for k in range(exp.macro_params.total_edges):
        i, j = edge_index[k]
        x1_f[k], y1_f[k], x2_f[k], y2_f[k] = fiber_end_coords(i, j)
    return x1_f, y1_f, x2_f, y2_f


def create_fiber_grid_plot(
    exp, file_code, edge_index, degrade_marker_frames, deg, tsave, x1_f, y1_f, x2_f, y2_f
):
    image_scale = (
        exp.macro_params.grid_node_distance
        / 72
        / (exp.macro_params.grid_node_distance - 10_000 * exp.macro_params.pore_size)
    )
    for sim in range(1):
        for idx, frame in enumerate(degrade_marker_frames[sim]):
            fig = plt.figure(
                figsize=(exp.macro_params.cols * image_scale, exp.macro_params.rows * image_scale), dpi=72
            )
            ax = fig.add_axes([0, 0, 1, 1])
            ax.set_axis_off()
            vmin = 0
            vmax = e.macro_params.cols - 1

            ydiff = e.macro_params.rows - e.macro_params.cols

            ax.set_xlim(vmin - 0.25, vmax + 0.25)
            ax.set_ylim(-vmax - ydiff - 0.25, -vmin + 0.25)
            ax.set_aspect("equal")
            fig.canvas.draw()

            # grid_width = 1000 * e.macro_params.grid_node_distance * (e.macro_params.cols - 1)  # in nanometers
            # figure_width = fig.get_figwidth() * fig.dpi  # in points (should be pixels too)
            # point_length = grid_width / figure_width  # in nanometers per point

            # fiber_thickness = (
            #     1000 * (e.macro_params.grid_node_distance - 10_000 * e.macro_params.pore_size) / point_length
            # )

            for k in range(e.macro_params.total_edges):
                i, j = edge_index[k]
                if j % 3 != 1:
                    if deg[sim][frame][k] > tsave[sim][frame]:
                        plt.plot(
                            [x1_f[k], x2_f[k]],
                            [-y1_f[k], -y2_f[k]],
                            color="black",
                            marker=None,
                            linewidth=1,
                        )
            fig.savefig(
                os.path.join(
                    e.os_path,
                    f"{sim:02}",
                    f"fiber_grid_plot{file_code[:-4]}_r{sim:02}_{degrade_percent_markers[idx]:.2f}.png",
                ),
                bbox_inches="tight",
            )
            plt.close()

In [ ]:
def calculate_time_row_exposed(exp, deg):
    exposed_time = np.empty(
        (exp.macro_params.total_trials, exp.macro_params.rows - 1, exp.macro_params.cols), dtype=np.float_
    )
    for run in range(exp.macro_params.total_trials):
        for j in range(exp.macro_params.cols):
            for i in range(exp.macro_params.rows - 1):
                if i == 0:
                    exposed_time[run, i, j] = 0
                else:
                    k = lysis.to_fortran_edge_index(i, j, exp.macro_params.rows, exp.macro_params.cols)
                    exposed_time[run, i, j] = max(exposed_time[run, i - 1, j], deg[run][-1, k])
    # exposed_time = 10* np.ceil(exposed_time / 10)
    return exposed_time / 60


def find_degradation_fronts(exp, exposed_time, y_distance, tsave):
    deg_fronts = []
    for r in range(exp.macro_params.total_trials):
        run_deg_fronts = []
        for j in range(exp.macro_params.cols):
            col_deg_front = []
            for i in range(1, exp.macro_params.rows - 1):
                if exposed_time[r, i - 1, j] < exposed_time[r, i, j] < tsave[r][-1] + 1:
                    col_deg_front.append([exposed_time[r, i, j], y_distance[i]])
            run_deg_fronts.append(np.array(col_deg_front).T)
        deg_fronts.append(run_deg_fronts)
    return deg_fronts


# TODO(bpaynter): Change this later to do mean and std of all columns across all runs
def mean_front_velocity(exp, deg_fronts):
    run_mean_velocity = np.empty(exp.macro_params.total_trials, dtype=np.float_)
    run_std_velocity = np.empty(exp.macro_params.total_trials, dtype=np.float_)
    for run in range(exp.macro_params.total_trials):
        front_velocity = np.empty(exp.macro_params.cols, dtype=np.float_)
        for j in range(exp.macro_params.cols):
            b, m = np.polynomial.polynomial.polyfit(deg_fronts[run][j][0], deg_fronts[run][j][1], 1)
            front_velocity[j] = m
        run_mean_velocity[run] = np.mean(front_velocity)
        run_std_velocity[run] = np.std(front_velocity)
    return np.mean(run_mean_velocity), np.mean(run_std_velocity)


def plot_front_degradation(exp, file_code, deg_fronts, deg):
    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_axis_on()
    ax.set_xlim(0, (max([np.max(deg_run[-1, :]) for deg_run in deg]) // 60) + 1)
    ax.set_ylim(
        (exp.macro_params.empty_rows - 1) * e.macro_params.grid_node_distance,
        (exp.macro_params.rows - 1) * exp.macro_params.grid_node_distance,
    )
    for run in range(exp.macro_params.total_trials):
        for j in range(exp.macro_params.cols):
            plt.plot(deg_fronts[run][j][0], deg_fronts[run][j][1], linewidth=1)
    fig.savefig(os.path.join(exp.os_path, "deg_fronts" + file_code[:-4] + ".png"), bbox_inches="tight")
    plt.close()


def mean_degradation_rate(exp, degraded_fraction, tsave):
    degradation_rate = np.empty(exp.macro_params.total_trials, dtype=np.float_)
    offset = np.empty(exp.macro_params.total_trials, dtype=np.float_)
    deg_start_time = np.empty(exp.macro_params.total_trials, dtype=np.float_)
    for r in range(exp.macro_params.total_trials):
        slope = np.empty(degraded_fraction[r].shape[0], dtype=np.float_)
        slope[0] = degraded_fraction[r][0]
        for t in range(1, degraded_fraction[r].shape[0]):
            slope[t] = degraded_fraction[r][t] - degraded_fraction[r][t - 1]
        degradation_happening = slope > slope.max() / 2
        s = np.argmax(degradation_happening)
        b, m = np.polynomial.polynomial.polyfit(
            tsave[r][degradation_happening] / 60, degraded_fraction[r][degradation_happening], 1
        )
        degradation_rate[r] = m
        offset[r] = b
        deg_start_time[r] = tsave[r][s] / 60
    return degradation_rate, offset, deg_start_time


def get_unbind_amounts(exp, file_code, mech):
    macro_unbind_pattern = re.compile(r"countmacrounbd=\s*(\d+)")
    micro_unbind_pattern = re.compile(r"countmicrounbd=\s*(\d+)")
    log_text = ""
    for sim in range(e.macro_params.total_trials):
        log_file_name = os.path.join(
            e.os_path, f"{sim:02}", f"{mech['executable']}{file_code[:-4]}_{sim:02}.txt"
        )
        with open(log_file_name, "r") as file:
            log_text += file.read()
    macro_unbinds = re.findall(macro_unbind_pattern, log_text)
    micro_unbinds = re.findall(micro_unbind_pattern, log_text)
    return np.array(macro_unbinds, dtype=int), np.array(micro_unbinds, dtype=int)


def get_processing_time(exp, file_code, mech):
    processing_time_pattern = re.compile(r"Processing time:\s*(\d+.\d+)\s*sec")
    log_text = ""
    for sim in range(e.macro_params.total_trials):
        log_file_name = os.path.join(
            e.os_path, f"{sim:02}", f"{mech['executable']}{file_code[:-4]}_{sim:02}.txt"
        )
        with open(log_file_name, "r") as file:
            log_text += file.read()
    processing_time = re.findall(processing_time_pattern, log_text)
    return np.array(processing_time, dtype=float)


def get_total_binds(exp, file_code, mech):
    total_binds_pattern = re.compile(r"Total Binds:\s*(\d+.\d+)\s*")
    log_text = ""
    for sim in range(e.macro_params.total_trials):
        log_file_name = os.path.join(
            e.os_path, f"{sim:02}", f"{mech['executable']}{file_code[:-4]}_{sim:02}.txt"
        )
        with open(log_file_name, "r") as file:
            log_text += file.read()
    total_binds = re.findall(total_binds_pattern, log_text)
    return np.array(total_binds, dtype=int)

In [ ]:
def ensure_results_match(exp, file_code, old_code):
    e_old = lysis.util.Experiment(
        os.path.join("..", "..", "data"), experiment_code=old_code + exp.experiment_code[-2:]
    )
    e_old.read_file()
    n_save, deg, tsave, mfpt, mol_location, mol_status = load_fortran_files(e, file_code)
    n_save_old, deg_old, tsave_old, mfpt_old, mol_location_old, mol_status_old = load_fortran_files(
        e_old, file_code
    )
    if np.count_nonzero(n_save != n_save_old) > 0:
        return False
    for i in range(e.macro_params.total_trials):
        if np.count_nonzero(deg[i] != deg_old[i]) > 0:
            return False
        if np.count_nonzero(tsave[i] != tsave_old[i]) > 0:
            return False
        if np.count_nonzero(mfpt[i] != mfpt_old[i]) > 0:
            return False
        if np.count_nonzero(mol_location[i] != mol_location_old[i]) > 0:
            return False
        if np.count_nonzero(mol_status[i] != mol_status_old[i]) > 0:
            return False
    return True


def read_mol_bind_events(exp, file_code):
    mol_bind_events = []
    for sim in range(e.macro_params.total_trials):
        filename = os.path.join(e.os_path, f"{sim:02}", f"m_bind_events{file_code[:-4]}_s{sim:02}.pkl")
        if os.path.isfile(filename):
            with open(filename, "rb") as handle:
                mol_bind_events.append(pickle.load(handle))
        else:
            raw_mol_bind_events = np.loadtxt(
                os.path.join(e.os_path, f"{sim:02}", f"m_bind_t{file_code[:-4]}_{sim:02}{file_code[-4:]}"),
                delimiter=",",
            )
            sim_mol_bind_events = [[] for _ in range(e.macro_params.total_molecules)]
            for event in raw_mol_bind_events:
                sim_mol_bind_events[event[1].astype(int) - 1].append((event[0], event[2].astype(int)))
            mol_bind_events.append(sim_mol_bind_events)
            with open(filename, "wb") as handle:
                pickle.dump(sim_mol_bind_events, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return mol_bind_events


def find_sim_dwell_transit_times(exp, sim_mol_bind_events, data):
    sim_times = {}
    for key in data:
        sim_times[key] = []
    for m in range(exp.macro_params.total_molecules):
        current_status = MolStatus.UNBOUND
        free_timer = 0.0
        total_timer = 0.0
        kinetic = True
        for event in sim_mol_bind_events[m]:
            if event[1] == MolStatus.BOUND:
                if current_status != MolStatus.UNBOUND:
                    raise RuntimeError("Binding when macro or micro or bound.")
                sim_times["Transit"].append((event[0], event[0] - total_timer, m))
                sim_times["Free Transit"].append((event[0], event[0] - free_timer, m))
                if kinetic:
                    sim_times["Kinetic Unbound"].append((event[0], event[0] - total_timer, m))
                kinetic = False
                total_timer = event[0]
            elif event[1] == MolStatus.UNBOUND:
                if current_status == MolStatus.UNBOUND:
                    raise RuntimeError("Unbinding when free.")
                match current_status:
                    case MolStatus.BOUND:
                        sim_times["Dwell"].append((event[0], event[0] - total_timer, m))
                    case MolStatus.MACRO_UNBOUND:
                        sim_times["Macro Unbound"].append((event[0], event[0] - total_timer, m))
                    case MolStatus.MICRO_UNBOUND:
                        sim_times["Micro Unbound"].append((event[0], event[0] - total_timer, m))
                    case _:
                        raise RuntimeError("Unreachable Case")
                if current_status == MolStatus.BOUND:
                    kinetic = True
                    total_timer = event[0]
            else:
                if current_status != MolStatus.BOUND:
                    raise RuntimeError("Unbinding when free or macro or micro.")
                sim_times["Dwell"].append((event[0], event[0] - total_timer, m))
                total_timer = event[0]
            current_status = event[1]
            free_timer = event[0]
    return sim_times


def save_dwell_transit_times(exp, file_code, mol_bind_events=None):
    if mol_bind_events is None:
        mol_bind_events = read_mol_bind_events(exp, file_code)
    data = ["Dwell", "Transit", "Free Transit", "Macro Unbound", "Micro Unbound", "Kinetic Unbound"]
    names = {
        "Dwell": "dwell_times",
        "Transit": "transit_times",
        "Free Transit": "free_transit_times",
        "Macro Unbound": "macro_unbound_times",
        "Micro Unbound": "micro_unbound_times",
        "Kinetic Unbound": "kinetic_transit_times",
    }
    for sim in range(exp.macro_params.total_trials):
        sim_times = find_sim_dwell_transit_times(e, mol_bind_events[sim], data)
        for key in data:
            columns = pd.Index(
                ["Simulation Time at Interval End (sec)", "Interval Length (sec)", "Molecule Index"]
            )
            df = pd.DataFrame(sim_times[key], columns=columns)
            df.iloc[:, [2, 0, 1]].to_csv(
                os.path.join(
                    exp.os_path,
                    f"{sim:02}",
                    f"m_{names[key]}{file_code[:-4]}_s{sim:02}.csv",
                ),
                index=False,
            )


def dwell_transit_times_by_interval(exp, mol_bind_events, marker_times):
    data = ["Dwell", "Transit", "Free Transit", "Macro Unbound", "Micro Unbound", "Kinetic Unbound"]
    times = {}
    for key in data:
        times[key] = [[] for _ in range(marker_times.shape[1] - 1)]
    for sim in range(exp.macro_params.total_trials):
        sim_times = find_sim_dwell_transit_times(exp, mol_bind_events[sim], data)
        for key in data:
            sim_times[key] = np.array(sim_times[key]).T / 60
            bins = np.searchsorted(marker_times[sim], sim_times[key][0])
            for idx, bin_num in enumerate(bins):
                if 1 <= bin_num <= marker_times.shape[1] - 1:
                    times[key][bin_num - 1].append(sim_times[key][:, idx])
    for key in data:
        for bin_num in range(marker_times.shape[1] - 1):
            times[key][bin_num] = np.array(times[key][bin_num]).T
    if marker_times.shape[1] == 2:
        for key in data:
            times[key] = times[key][0]
    return times


def transit_time_medians(exp, deg_fraction, mol_bind_events):
    degrade_percent_markers = np.arange(0.2, 0.8, 0.05)
    degrade_marker_times = np.empty(
        (exp.macro_params.total_trials, len(degrade_percent_markers)), dtype=float
    )
    for sim in range(exp.macro_params.total_trials):
        for marker in range(len(degrade_percent_markers)):
            degrade_marker_frame = np.argmax(deg_fraction[sim] >= degrade_percent_markers[marker])
            degrade_marker_times[sim, marker] = tsave[sim][degrade_marker_frame] / 60
    times = dwell_transit_times(e, mol_bind_events, degrade_marker_times)
    medians = np.empty(degrade_marker_times.shape[1] - 1, dtype=float)
    for idx, array in enumerate(times["Transit"]):
        medians[idx] = np.median(array[1])
    return medians

###########################################################################
## Maybe try diagonal lines, since that is how the tPA move in the model ##
###########################################################################
def get_pore_stats(exp, deg, tsave):
    rev_edge_index = get_rev_edge_index(e)
    num_of_lines = 10  # 25
    v_step = (exp.macro_params.cols - 1) // num_of_lines
    v_lines = range(v_step // 2, exp.macro_params.cols - 1, v_step)
    h_step = (exp.macro_params.rows - 1) // num_of_lines
    h_lines = range(h_step // 2, exp.macro_params.rows - 1, h_step)
    min_time = np.min(n_save)
    pores = []
    for sim in range(e.macro_params.total_trials):
        pores.append([])
        for frame in range(len(tsave[sim])):
            lines = []
            degraded = deg[sim][frame] < tsave[sim][frame]
            for j in v_lines:
                pores_in_line = []
                current_pore = exp.macro_params.pore_size * 10_000  # exp.macro_params.grid_node_distance  #
                if degraded[rev_edge_index[0, j * 3 + 2]]:
                    current_pore += exp.macro_params.grid_node_distance - exp.macro_params.pore_size * 10_000
                for i in range(1, exp.macro_params.rows - 1):
                    if degraded[rev_edge_index[i, j * 3 + 2]]:
                        current_pore += exp.macro_params.grid_node_distance
                    else:
                        pores_in_line.append(current_pore)
                        current_pore = (
                            exp.macro_params.pore_size
                            * 10_000
                            # exp.macro_params.grid_node_distance
                        )
                if degraded[rev_edge_index[-1, j * 3 + 2]]:
                    current_pore += exp.macro_params.grid_node_distance - exp.macro_params.pore_size * 10_000
                pores_in_line.append(current_pore)
                lines.append(np.array(pores_in_line))
            for i in h_lines:
                pores_in_line = []
                current_pore = exp.macro_params.pore_size * 10_000  # exp.macro_params.grid_node_distance #
                if degraded[rev_edge_index[i, 0]]:
                    current_pore += exp.macro_params.grid_node_distance - exp.macro_params.pore_size * 10_000
                for j in range(1, exp.macro_params.cols - 1):
                    if degraded[rev_edge_index[i, j * 3]]:
                        current_pore += exp.macro_params.grid_node_distance
                    else:
                        pores_in_line.append(current_pore)
                        current_pore = (
                            exp.macro_params.pore_size
                            * 10_000
                            # exp.macro_params.grid_node_distance
                        )
                if degraded[rev_edge_index[i, -1]]:
                    current_pore += exp.macro_params.grid_node_distance - exp.macro_params.pore_size * 10_000
                pores_in_line.append(current_pore)
                lines.append(np.array(pores_in_line))
            pores_at_frame = np.empty(0, dtype=float)
            for line in lines:
                pores_at_frame = np.append(pores_at_frame, line)
            pores[sim].append(
                pores_at_frame
                # / (exp.macro_params.pore_size * 10_000)
                # / exp.macro_params.grid_node_distance
            )
    return pores


def save_pore_stats(exp, file_code, tsave, pores, deg_fraction):
    for sim in range(exp.macro_params.total_trials):
        index = pd.Index(tsave[sim] / 60, name="Time Elapsed (min)")
        df = pd.DataFrame(pores[sim], index=index)
        df.insert(0, "Fraction Degraded", deg_fraction[sim])
        df.to_csv(
            os.path.join(
                exp.os_path,
                f"{sim:02}",
                f"pore_sizes{file_code[:-4]}_s{sim:02}.csv",
            )
        )


def plot_pore_size(exp, tsave, pores_by_frame):
    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_xlim(0, (max([np.max(deg_run[-1, :]) for deg_run in deg]) // 60) + 1)
    # ax.set_ylim(-0.1, 1.1)
    for sim in range(exp.macro_params.total_trials):
        plt.plot(tsave[sim] / 60, degraded_fraction[sim])
        for slope in range(len(degrade_slope_between_percent)):
            start_frame = degrade_marker_frames[
                sim, degrade_percent_markers.index(degrade_slope_between_percent[slope][0])
            ]
            end_frame = degrade_marker_frames[
                sim, degrade_percent_markers.index(degrade_slope_between_percent[slope][1])
            ]
            lin_func = (
                lambda x: degradation_rates[sim, slope] * (x - tsave[sim][start_frame] / 60)
                + degraded_fraction[sim][start_frame]
            )
            plt.plot(
                tsave[sim][start_frame:end_frame] / 60,
                lin_func(tsave[sim][start_frame:end_frame] / 60),
                color="b",
                alpha=0.5,
                zorder=0.1,
            )
    fig.savefig(os.path.join(exp.os_path, "deg_rate" + file_code[:-4] + ".png"), bbox_inches="tight")
    plt.close()

In [ ]:
pore_plot_data = {}
transit_time_data = {}
for run in tqdm(runs):
    # Select and load scenario
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(os.path.join("..", "..", "data"), experiment_code=run["exp_code"])
    # e = lysis.util.Experiment(
    #     os.path.join("/", "home", "bpaynter", "Archive", "lysis_data"), experiment_code=run["exp_code"]
    # )
    e.read_file()
    y_distance = np.arange(e.macro_params.rows - 1) * e.macro_params.grid_node_distance
    file_code = out_file_code.format(scenario_code=run["scenario"]) + ".dat"

    # Get processing time and print
    # time = get_processing_time(e, file_code, mech).mean()
    # time = str(datetime.timedelta(seconds=time)).split(".")[0]
    # print(run["exp_code"], file_code[1:-4], f" - Mean Simulation Processing Time: {time}")

    # Read data from log files
    # total_binds = get_total_binds(e, file_code, mech)
    # macro_unbinds, micro_unbinds = get_unbind_amounts(e, file_code, mech)
    # normal_unbinds = total_binds - (macro_unbinds + micro_unbinds)
    # mol_time_df.loc[
    #     scen["descriptor"], "Total Binds"
    # ] = f"{total_binds.mean():,.0f} \u00B1 {total_binds.std():,.0f}"
    # mol_time_df.loc[scen["descriptor"], "Macro Unbinds %"] = f"{(macro_unbinds / total_binds).mean():.3%}"
    # mol_time_df.loc[scen["descriptor"], "Micro Unbinds %"] = f"{(micro_unbinds / total_binds).mean():.3%}"

    # Read data from data files
    n_save, deg, tsave, mfpt, mol_location, mol_status = load_fortran_files(e, file_code)

    # Check that data matches earlier experiment group
    # old_code = "2023-05-23-10"
    # if not ensure_results_match(e, file_code, old_code):
    #     print("ERROR")

    # Read parameters into a dataframe
    # params = e.macro_params
    # df = pd.DataFrame([params])
    # parameters_df.loc[scen["descriptor"]] = df.loc[0]
    # if e.macro_params.total_time == 0:
    #     parameters_df.loc[scen["descriptor"], "total_time"] = max([i[-1] for i in tsave])
    #     parameters_df.loc[scen["descriptor"], "total_time_steps"] = max(
    #         [int(i[-1] // e.macro_params.time_step) for i in tsave]
    #     )
    #     parameters_df.loc[scen["descriptor"], "number_of_saves"] = max(n_save)

    # binds_df.loc[scen["descriptor"]] = get_total_binds(e, file_code, mech)

    # Process results
    deg_fraction = find_degraded_fraction(e, deg, tsave)
    marker_frames = find_degradation_marker_frames(e, deg_fraction)
    marker_times = find_degradation_marker_times(e, marker_frames, tsave)
    # degradation_time_df.loc[run["scenario"]] = marker_times.T.flatten()
    # slopes = degradation_rates(e, marker_frames, deg_fraction, tsave)
    # slopes_df.loc[run["scenario"]] = slopes.T.flatten()

    #     save_degrade_fraction_table(e, file_code, deg_fraction, tsave)

    #     mol_bind_events = read_mol_bind_events(e, file_code)
    #     # save_dwell_transit_times(e, file_code, mol_bind_events)
    #     times = dwell_transit_times_by_interval(e, mol_bind_events, marker_times[:, [1, 3]])

    #     for key, value in times.items():
    #         mol_time_df.loc[
    #             scen["descriptor"], key + " Time (min)"
    #         ] = f"{value[1].mean():.4f} \u00B1 {value[1].std():.4f}"

    #     # dwell_time_data[scen["descriptor"]] = dwell_times
    #     #

    #     #     mol_time_df.loc[
    #     #         scen["descriptor"], "Total Binds"
    #     #     ] = f"{times['Dwell'].shape[1]/e.macro_params.total_trials:,.0f}"
    #     #     mol_time_df.loc[
    #     #         scen["descriptor"], "Macro Unbinds %"
    #     #     ] = f"{times['Macro Unbound'].shape[1] / times['Dwell'].shape[1]:.3%}"
    #     #     mol_time_df.loc[
    #     #         scen["descriptor"], "Micro Unbinds %"
    #     #     ] = f"{times['Micro Unbound'].shape[1] / times['Dwell'].shape[1]:.3%}"

    # times = None

    #     #     # transit_time_data[scen["descriptor"]] = transit_time_medians(e, deg_fraction, mol_bind_events)

    # mol_bind_events = None

    pore_data = get_pore_stats(e, deg, tsave)

    #     # save_pore_stats(e, file_code, tsave, pore_data, deg_fraction)
    marker_pores = {}
    #     # pore_plot_data[scen["descriptor"]] = {}
    for idx, percent in enumerate(degrade_percent_markers):
        marker_pores[percent] = np.concatenate(
            [pore_data[sim][marker_frames[sim][idx]] for sim in range(e.macro_params.total_trials)]
        )
        # pore_plot_data[scen["descriptor"]][percent] = [
        #     pore_data[sim][marker_frames[sim][idx]] for sim in range(e.macro_params.total_trials)
        # ]
        pore_stat_df.loc[
            scen["descriptor"], f"{int(percent*100)}%"
        ] = f"{marker_pores[percent].mean():.3f} \u00B1 {marker_pores[percent].std():.3f}"
#     plot_means = np.array(
#         [np.mean(row) for row in pore_data[0][marker_frames[0][1] : marker_frames[0][3] + 1]], dtype=float
#     )
#     plot_stds = np.array(
#         [np.std(row) for row in pore_data[0][marker_frames[0][1] : marker_frames[0][3] + 1]], dtype=float
#     )
#     pore_plot_data[scen["descriptor"]] = (
#         deg_fraction[0][marker_frames[0][1] : marker_frames[0][3] + 1],
#         plot_means,
#         plot_stds,
#     )

#     plot_degradation_percent(e, deg_fraction, tsave, marker_frames, slopes)

#     edge_index = get_edge_index(e)
#     x_f, y_f, x_m, y_m = animation_data(e, edge_index, mol_location)
#     create_animation(e, file_code, edge_index, deg, mol_status, tsave, x_f, y_f, x_m, y_m)
#     create_animation_stills(
#         e, file_code, marker_frames, edge_index, deg, mol_status, tsave, x_f, y_f, x_m, y_m
#     )

#     x1_f, y1_f, x2_f, y2_f = grid_data(e, edge_index)
#     create_fiber_grid_plot(e, file_code, edge_index, marker_frames, deg, tsave, x1_f, y1_f, x2_f, y2_f)

In [ ]:
scenario_grid = [
    ["TN-L_9350", "TK-L_3042", "TN-D_684", "TK-D_307"],
    ["TN-L_9350", "TK-L_9350", "TN-D_9350", "TK-D_9350"],
    ["TN-L_307", "TK-L_307", "TN-D_307", "TK-D_307"],
]
scenario_grid_T = [
    ["TN-L_9350", "TN-L_9350", "TN-L_307"],
    ["TN-D_684", "TN-D_9350", "TN-D_307"],
    ["TK-L_3042", "TK-L_9350", "TK-L_307"],
    ["TK-D_307", "TK-D_9350", "TK-D_307"],
]
groupings = [
    "Fixed Concentration",
    "Fixed Ratio - High",
    "Fixed Ratio - Low",
]
colors = {
    "TN-L_9350": "xkcd:red",
    "TN-L_307": "xkcd:dark purple",
    "TN-D_684": "xkcd:green",
    "TN-D_9350": "xkcd:violet",
    "TN-D_307": "xkcd:orange",
    "TK-L_3042": "xkcd:black",
    "TK-L_9350": "xkcd:brown",
    "TK-L_307": "xkcd:navy blue",
    "TK-D_307": "xkcd:cyan",
    "TK-D_9350": "xkcd:maroon",
}

In [ ]:
# print(mol_time_df.to_html())
# print(
#     mol_time_df[
#         [
#             # "Total Binds",
#             # "Macro Unbinds %",
#             # "Micro Unbinds %",
#             "Dwell Time (min)",
#             "Transit Time (min)",
#             # "Kinetic Transit Time (min)",
#             # "Free Transit Time (min)",
#             # "Macro-unbound Time (min)",
#             # "Micro-unbound Time (min)",
#         ]
#     ].to_markdown()
# )
print(pore_stat_df.to_markdown())

In [ ]:
rng1 = np.random.default_rng()

In [ ]:
##################################
## Generate pores with random degradation
##################################
exp = None
random_pore_data = {}
trials = 10_000
percent = 0.8
for run in [r for r in runs if r["scenario"][-3:] == "307"]:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(os.path.join("..", "..", "data"), experiment_code=run["exp_code"])
    # e = lysis.util.Experiment(
    #     os.path.join("/", "home", "bpaynter", "Archive", "lysis_data"), experiment_code=run["exp_code"]
    # )
    e.read_file()
    total = e.macro_params.total_fibers
    rev_edge_index = get_rev_edge_index(e)
    num_of_lines = 25
    v_step = (e.macro_params.cols - 1) // num_of_lines
    v_lines = range(v_step // 2, e.macro_params.cols - 1, v_step)
    h_step = (e.macro_params.rows - 1) // num_of_lines
    h_lines = range(h_step // 2, e.macro_params.rows - 1, h_step)
    min_time = np.min(n_save)
    pores = []
    for sim in trange(trials):
        pores.append([])
        lines = []
        remove = rng1.choice(np.arange(total), size=math.floor(total * percent), replace=False)
        degraded = np.full(total, False)
        for i in remove:
            degraded[i] = True
        for j in v_lines:
            pores_in_line = []
            current_pore = e.macro_params.grid_node_distance  # e.macro_params.pore_size * 10_000
            # if degraded[rev_edge_index[0, j * 3 + 2]]:
            #     current_pore += exp.macro_params.grid_node_distance - exp.macro_params.pore_size * 10_000
            for i in range(1, e.macro_params.rows - 1):
                if degraded[rev_edge_index[i, j * 3 + 2]]:
                    current_pore += e.macro_params.grid_node_distance
                else:
                    pores_in_line.append(current_pore)
                    current_pore = e.macro_params.grid_node_distance  # e.macro_params.pore_size * 10_000
            # if degraded[rev_edge_index[-1, j * 3 + 2]]:
            #     current_pore += exp.macro_params.grid_node_distance - exp.macro_params.pore_size * 10_000
            pores_in_line.append(current_pore)
            lines.append(np.array(pores_in_line))
        for i in h_lines:
            pores_in_line = []
            current_pore = e.macro_params.grid_node_distance  # e.macro_params.pore_size * 10_000
            # if degraded[rev_edge_index[i, 0]]:
            #     current_pore += exp.macro_params.grid_node_distance - exp.macro_params.pore_size * 10_000
            for j in range(1, e.macro_params.cols - 1):
                if degraded[rev_edge_index[i, j * 3]]:
                    current_pore += e.macro_params.grid_node_distance
                else:
                    pores_in_line.append(current_pore)
                    current_pore = e.macro_params.grid_node_distance  # e.macro_params.pore_size * 10_000
            # if degraded[rev_edge_index[i, -1]]:
            #     current_pore += exp.macro_params.grid_node_distance - exp.macro_params.pore_size * 10_000
            pores_in_line.append(current_pore)
            lines.append(np.array(pores_in_line))
        pores_at_frame = np.empty(0, dtype=float)
        for line in lines:
            pores_at_frame = np.append(pores_at_frame, line)
        pores[sim].append(
            pores_at_frame / e.macro_params.grid_node_distance
        )  # (e.macro_params.pore_size * 10_000))
    random_pore_data[scen["descriptor"]] = [pores[sim][0] for sim in range(trials)]

In [ ]:
fig, axs = plt.subplots(
    nrows=1,
    ncols=4,
    sharex=False,
    sharey=True,
    figsize=(12, 6),
    constrained_layout=True,
    # width_ratios=[2, 1, 1, 1, 1],
)
for i, row in enumerate(scenario_grid):
    axs[0].set_ylabel("pore size\n(as a multiple of grid node distance")  # initial pore size)")
    # for j, scenario in enumerate(row):
    #     axs[i, j + 2].boxplot(
    #         [pore_plot_data[scenario][0.8][sim] for sim in range(10)],
    #         # showfliers=False,
    #         whis=[0.01, 0.99],
    #     )
    #     # axs[i].set_xticks(
    #     #     range(1, len(row) + 1),
    #     #     labels=[f"{scenario}" for scenario in row],
    #     # )
    #     axs[i, j + 2].set_title(scenario)
    axs[i].boxplot(
        [np.concatenate(pore_plot_data[scenario][percent]) for scenario in row],
        showfliers=False,
        whis=(1, 99),
    )
    axs[i].set_xticks(
        range(1, len(row) + 1),
        labels=[f"{scenario}" for scenario in row],
        rotation=-45,
    )
    axs[i].set_title(groupings[i])
    axs[i].grid(visible=True, which="major", axis="y")
    # axs[i, 1].boxplot(
    #     random_pore_data[f"{row[0][:5]}307"],
    #     showfliers=False,
    # )
    # axs[i, 1].set_title(f"{row[0][:5]}Control")
axs[-1].boxplot(
    [np.concatenate(random_pore_data[f"{scenario[:5]}307"]) for scenario in scenario_grid[0]],
    showfliers=False,
    whis=(1, 99),
)
axs[-1].set_xticks(
    range(1, len(row) + 1),
    labels=[f"{scenario[:5]}Control" for scenario in scenario_grid[0]],
    rotation=-45,
)
axs[-1].set_title("Random Control")
axs[-1].grid(visible=True, which="major", axis="y")
fig.suptitle(
    f"Distribution of pore sizes at {percent:.0%} degradation.\n(Boxes are Q1 to Q3; Whiskers are 1st percentile to 99th percentile)"
)
pass

In [ ]:
# 10_000 samples
# 80% degraded
for run in runs:
    print(
        run["scenario"],
        scipy.stats.ks_2samp(
            np.concatenate(pore_plot_data[run["scenario"]][0.8]),
            np.concatenate(random_pore_data[f"{run['scenario'][:5]}307"]),
        ),
    )

In [ ]:
# 100 samples
for run in runs:
    print(
        run["scenario"],
        scipy.stats.ks_2samp(
            np.concatenate(pore_plot_data[run["scenario"]][0.8]),
            np.concatenate(random_pore_data[f"{run['scenario'][:5]}307"]),
        ),
    )

In [ ]:
scipy.stats.anderson(np.concatenate(random_pore_data[f"TK-L_307"]), dist="expon")

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=4, sharex=False, sharey=True, figsize=(12, 6), constrained_layout=True)
for i, row in enumerate(scenario_grid_T):
    axs[i].set_yscale("log")
    for j, scenario in enumerate(row):
        combined_sims = np.concatenate(pore_plot_data[scenario][percent])
        hist = np.array([(i, np.count_nonzero(combined_sims == i)) for i in np.unique(combined_sims)])
        axs[i].plot(
            hist.T[0], hist.T[1] * 100 / combined_sims.shape[0], color=colors[scenario], label=scenario
        )
    combined_sims = np.concatenate(random_pore_data[f"{row[0][:5]}307"])
    hist = np.array([(i, np.count_nonzero(combined_sims == i)) for i in np.unique(combined_sims)])
    axs[i].plot(hist.T[0], (hist.T[1]) * 100 / combined_sims.shape[0], label=f"{row[0][:5]}Control")
    axs[i].set_xlabel("Pore size (in multiples of grid distance)")
    axs[i].legend()
axs[0].set_ylabel("Percent of pores (log scale)")
fig.suptitle(f"Distribution of pore sizes at {percent:.0%} degradation.")
pass

In [ ]:
fig, axs = plt.subplots(
    nrows=4, ncols=1, sharex=False, sharey=False, figsize=(12, 12), constrained_layout=True
)
# ax.scatter(transit_times[0], transit_times[1], marker="o", s=(72.0 / fig.dpi) ** 2, c="xkcd:red")
axs[0].scatter(
    macro_unbound_times[0], macro_unbound_times[1], marker="o", s=(72.0 / fig.dpi) ** 2, c="xkcd:green"
)
axs[0].set_title("Macro-Unbinding")
axs[1].scatter(
    micro_unbound_times[0], micro_unbound_times[1], marker="o", s=(72.0 / fig.dpi) ** 2, c="xkcd:blue"
)
axs[1].set_title("Micro-Unbinding")
axs[2].scatter(
    kinetic_unbound_times[0], kinetic_unbound_times[1], marker="o", s=(72.0 / fig.dpi) ** 2, c="xkcd:red"
)
axs[2].set_title("Kinetic-Unbinding")
axs[3].scatter(
    macro_unbound_times[0], macro_unbound_times[1], marker="o", s=(72.0 / fig.dpi) ** 2, c="xkcd:green"
)
axs[3].set_title("All Molecules")
axs[3].scatter(
    micro_unbound_times[0], micro_unbound_times[1], marker="o", s=(72.0 / fig.dpi) ** 2, c="xkcd:blue"
)
axs[3].scatter(
    kinetic_unbound_times[0], kinetic_unbound_times[1], marker="o", s=(72.0 / fig.dpi) ** 2, c="xkcd:red"
)


for i in range(4):
    axs[i].set_ylabel("Molecule Transit Time (min)")
    axs[i].set_xticks(marker_times[0][1:], ["20%", "50%", "80%"])
    axs[i].set_xlabel("Fraction of Fibers Degraded")
axs[0].axhline(1, color=(0, 0, 0))
axs[1].set_ylim(-0.1, 1)
# fig.suptitle("Time from unbinding to rebinding")

In [ ]:
units = lysis.util.MacroParameters.units()
rename = {}
for k, v in units.items():
    rename[k] = f"{k} ({v})"
printable_parameters_df = pd.DataFrame(index=parameters_df.index)
for col in parameters_df.columns:
    if not col in cols_to_drop:
        if col in formats:
            printable_parameters_df[col] = parameters_df[col].apply(lambda x: formats[col].format(x))
        else:
            printable_parameters_df[col] = parameters_df[col]
printable_parameters_df = printable_parameters_df.rename(columns=rename)
print(printable_parameters_df.T.to_markdown())

In [ ]:
m = slopes_df.groupby(axis=1, level=0).mean()
s = slopes_df.groupby(axis=1, level=0).std()
slopes_summary = pd.DataFrame(index=slopes_df.index, columns=slopes_df.columns.get_level_values(0).unique())
for i in slopes_summary.index:
    for j in slopes_summary.columns:
        slopes_summary.loc[i, j] = f"{m.loc[i, j]:.6f} \u00B1 {s.loc[i, j]:.6f}"
# print(slopes_summary.to_html())
print(slopes_summary.to_markdown())

In [ ]:
m = degradation_time_df.groupby(axis=1, level=0).mean()
s = degradation_time_df.groupby(axis=1, level=0).std()
degradation_time_summary = pd.DataFrame(
    index=degradation_time_df.index, columns=degradation_time_df.columns.get_level_values(0).unique()
)
for i in degradation_time_summary.index:
    for j in degradation_time_summary.columns:
        degradation_time_summary.loc[i, j] = f"{m.loc[i, j]:.2f} \u00B1 {s.loc[i, j]:.2f}"
print(degradation_time_summary.to_markdown())
# degradation_time_summary

In [ ]:
run = runs[0]
sim = 0
scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
scen

In [ ]:
mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
e = lysis.util.Experiment(os.path.join("..", "..", "data"), experiment_code=run["exp_code"])
# e = lysis.util.Experiment(
#     os.path.join("/", "home", "bpaynter", "Archive", "lysis_data"), experiment_code=run["exp_code"]
# )
e.read_file()
y_distance = np.arange(e.macro_params.rows - 1) * e.macro_params.grid_node_distance
file_code = out_file_code.format(scenario_code=run["scenario"]) + ".dat"
print(run["exp_code"], file_code)
n_save, deg, tsave, mfpt, mol_location, mol_status = load_fortran_files(e, file_code)

deg_fraction = find_degraded_fraction(e, deg, tsave)
marker_frames = find_degradation_marker_frames(e, deg_fraction)
marker_times = find_degradation_marker_times(e, marker_frames, tsave)
edge_index = get_edge_index(e)
x1_f, y1_f, x2_f, y2_f = grid_data(e, edge_index)

In [ ]:
mol_bind_events = []
for sim in range(e.macro_params.total_trials):
    raw_mol_bind_events = np.loadtxt(
        os.path.join(e.os_path, f"{sim:02}", f"m_bind_t{file_code[:-4]}_{sim:02}{file_code[-4:]}"),
        delimiter=",",
    )
    sim_mol_bind_events = [[] for _ in range(e.macro_params.total_molecules)]
    for event in raw_mol_bind_events:
        sim_mol_bind_events[event[1].astype(int) - 1].append((event[0], event[2].astype(int)))
    mol_bind_events.append(sim_mol_bind_events)

In [ ]:
for sim in range(e.macro_params.total_trials):
    for m in range(e.macro_params.total_molecules):
        for event in range(len(mol_bind_events[sim][m]) - 1):
            if mol_bind_events[sim][m][event][1] == 0:
                if mol_bind_events[sim][m][event + 1][1] != 1:
                    print("ERROR: Full unbind NOT followed by bind!")
            if mol_bind_events[sim][m][event][1] == 1:
                if mol_bind_events[sim][m][event + 1][1] == 1:
                    print("ERROR: Two binds in a row!")
            if mol_bind_events[sim][m][event][1] == 2:
                if mol_bind_events[sim][m][event + 1][1] != 0:
                    print("ERROR: Macro-unbind NOT followed by full unbind!")
            if mol_bind_events[sim][m][event][1] == 3:
                if mol_bind_events[sim][m][event + 1][1] != 0:
                    print("ERROR: Macro-unbind NOT followed by full unbind!")

In [ ]:
dwell_times = []
transit_times = []
for sim in range(e.macro_params.total_trials):
    for m in range(e.macro_params.total_molecules):
        current_status = MolStatus.UNBOUND
        timer = 0.0
        for event in mol_bind_events[sim][m]:
            if current_status == MolStatus.UNBOUND and event[1] == MolStatus.BOUND:
                transit_times.append(event[0] - timer)
                timer = event[0]
                current_status = MolStatus.BOUND
            if current_status == MolStatus.BOUND and event[1] == MolStatus.UNBOUND:
                dwell_times.append(event[0] - timer)
                timer = event[0]
                current_status = MolStatus.UNBOUND
            # if current_status == MolStatus.BOUND and event[1] == MolStatus.MICRO_UNBOUND:
            #     # dwell_times.append(event[0] - timer)
            #     timer = event[0]
            #     current_status = MolStatus.UNBOUND

In [ ]:
np.array(dwell_times).mean(), np.array(dwell_times).std()

In [ ]:
np.array(transit_times).mean(), np.array(transit_times).std()

In [ ]:
binds = 0
macro_unbinds = 0
micro_unbinds = 0
for m in range(e.macro_params.total_molecules):
    for event in mol_bind_events[0][m]:
        if event[1] == MolStatus.BOUND:
            binds += 1
        if event[1] == MolStatus.MACRO_UNBOUND:
            macro_unbinds += 1
        if event[1] == MolStatus.MICRO_UNBOUND:
            micro_unbinds += 1
binds, macro_unbinds, micro_unbinds

In [ ]:
len(dwell_times)

In [ ]:
print(f"At {percent:.0%} degradation, percent of pores at initial size.")
for run in runs:
    a = (
        np.count_nonzero(np.concatenate(pore_plot_data[run["scenario"]][percent]) == 1)
        / np.concatenate(pore_plot_data[run["scenario"]][0.8]).size
    )
    b = (
        np.count_nonzero(np.concatenate(random_pore_data[f"{run['scenario'][:5]}307"]) == 1)
        / np.concatenate(random_pore_data[f"{run['scenario'][:5]}307"]).size
    )
    print(f"{run['scenario']}: Expected {b:.2%}; Actual {a:.2%}")

In [ ]:
(np.arange(30) * exp.macro_params.grid_node_distance + exp.macro_params.pore_size * 10_000) / (
    exp.macro_params.pore_size * 10_000
)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, sharex=False, sharey=True, figsize=(12, 6), constrained_layout=True)
axs.boxplot([random_pore_data[scenario] for scenario in random_pore_data.keys()], showfliers=False)
axs.set_xticks(
    range(1, len(random_pore_data) + 1),
    labels=[f"{scenario}" for scenario in random_pore_data.keys()],
)
pass

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, sharex=False, sharey=True, figsize=(12, 6), constrained_layout=True)
axs.boxplot([pore_plot_data[scenario][0.5] for scenario in pore_plot_data.keys()], showfliers=False)
axs.set_xticks(
    range(1, len(pore_plot_data) + 1),
    labels=[f"{scenario}" for scenario in pore_plot_data.keys()],
)

In [ ]:
random_pore_data["TN-L_307"].mean(), random_pore_data["TN-L_9350"].mean()

In [ ]:
random_pore_data["TK-L_307"]

In [ ]:
plt.boxplot([pore_data[sim][marker_frames[sim][3]] for sim in range(10)], showfliers=False)

In [ ]:
plt.boxplot(
    [random_pore_data["TK-D_307"]] + pore_plot_data["TK-D_307"] + pore_plot_data["TK-D_9350"],
    showfliers=False,
)
pass

In [ ]:
type(pore_plot_data["TK-D_307"])